In [32]:
# Run the code below each time the notebook is started or restarted to ensure
# that if you change any code in the library, this notebook will use the latest
# version of the library code.
%load_ext autoreload
%autoreload 2

# contains scraping and cleaning functions
from helpers import *
# setting display settings so that the maximum amount of data will show
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

import pandas as pd # library for data analysis
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML documents
# get the response in the form of html

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
wikiurl = "https://en.wikipedia.org/wiki/2004_Summer_Olympics"
index = 1
output_file = "data/athletes-2004.csv"

table_class = "wikitable sortable jquery-tablesorter"
response = requests.get(wikiurl)
# status code must be 200 to legally scrape
if response.status_code == 200:
    # Parse data from the html into a beautifulsoup object
    # soup = BeautifulSoup(response.text, "html.parser")
    # tables=soup.findAll("table",{"class": "wikitable"})
    # df=pd.read_html(str(tables[index]))
    # # Convert list to dataframe
    # df=pd.DataFrame(df[0])
    # return df
    
    whole_page = BeautifulSoup(response.text, "html.parser")
    tables = whole_page.findAll("table",{"class": "wikitable"})
    soup = tables[index]
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = ';'.join(chunk for chunk in chunks if chunk)

    print(text)
    
    
else:
    print("Error: This table should not be scraped due to its status" \
          " code.")

Participating National Olympic Committees;Afghanistan (5 athletes);Albania (7);Algeria (61);American Samoa (3);Andorra (6);Angola (30);Antigua and Barbuda (5);Argentina (152);Armenia (18);Aruba (4);Australia (470);Austria (74);Azerbaijan (36);Bahamas (22);Bahrain (10);Bangladesh (4);Barbados (10);Belarus (151);Belgium (50);Belize (2);Benin (4);Bermuda (10);Bhutan (2);Bolivia (7);Bosnia and Herzegovina (9);Botswana (11);Brazil (243);British Virgin Islands (1);Brunei (1);Bulgaria (95);Burkina Faso (5);Burundi (7);Cambodia (4);Cameroon (17);Canada (263);Cape Verde (3);Cayman Islands (5);Central African Republic (4);Chad (1);Chile (22);China (384);Colombia (53);Comoros (3);Republic of the Congo (5);Cook Islands (3);Costa Rica (20);Croatia (81);Cuba (151);Cyprus (20);Czech Republic (142);Democratic Republic of the Congo (4);Denmark (92);Dominica (2);Dominican Republic (33);Ecuador (16);Egypt (97);El Salvador (8);Equatorial Guinea (2);Eritrea (4);Estonia (42);Ethiopia (26);Federated States o

In [41]:
data = text.encode('utf-8')
udata=data.decode("utf-8")
all_text=str(udata.encode("ascii","ignore"))
# print(all_text)

comma_separated = all_text.replace("(",",")
comma_separated = comma_separated.replace(")","")
comma_separated = comma_separated.replace("b'Participating National Olympic Committees;","")
comma_separated = comma_separated.replace("'","")
comma_separated = comma_separated.replace(" (host)","")
comma_separated = comma_separated.replace(" athletes","")
comma_separated = comma_separated.replace(" [144]","")
comma_separated = comma_separated.replace("[143]","")
comma_separated = comma_separated.replace("[144]","")
# print(comma_separated)
# print(text.encode('utf-8'))

data = comma_separated
df = pd.DataFrame([x.split(',') for x in data.split(';')])
df.drop(2, axis = 1, inplace = True)
df.rename(columns = {0: "Country", 1: "Athletes"}, inplace = True)
df

,Country,Athletes
0,Afghanistan,5
1,Albania,7
2,Algeria,61
3,American Samoa,3
4,Andorra,6
5,Angola,30
6,Antigua and Barbuda,5
7,Argentina,152
8,Armenia,18
9,Aruba,4


In [42]:
# df.to_csv(output_file, index=False)